# How to work with CMEMS reanalysis on Earth Data Hub
### Analysis of el Nino events

***
This notebook will provide you guidance on how to access and use the `cmems_mod_glo_phy_my_0.083_P1D-m.zarr` datset on Earth Data Hub.

The first goal is to compute the average anomaly of the Sea Surface Height over the Pacific equator region during the 1997-1998 El Nino event. We will use the whole time range provided by the dataset as a reference period.

The second goal is to highlight other occurences of El Nino in the available time span.
***

## What you will learn:

* how to access and preview the dataset
* select and reduce the data
* plot the results

## Data access and preview
***

Xarray and Dask work together following a lazy principle. This means when you access and manipulate a Zarr store the data is in not immediately downloaded and loaded in memory. Instead, Dask constructs a task graph that represents the operations to be performed. A smart user will reduce the amount of data that needs to be downloaded before the computation takes place (e.g., when the `.compute()` or `.plot()` methods are called).

To preview the data, only the dataset metadata must be downloaded. Xarray does this automatically:

***

In [ ]:
import xarray as xr

# your `~/.netrc` file MUST contain your credentials for earthdatahub.com
#
# machine earthdatahub.com
#   login {your_username}
#   password {your_password}

ds = xr.open_dataset(
    "https://earthdatahub.com/stores/cmems-reanalysis/cmems_mod_glo_phy_my_0.083_P1D-m.zarr",
    chunks={},
    engine="zarr",
    storage_options={"client_kwargs": {"trust_env": True}},
)
ds

## Working with data

Datasets on EDH are typically very large and remotely hosted. Typical use imply a selection of the data followed by one or more reduction steps to be performed in a local or distributed Dask environment. 

The structure of a workflow that uses EDH data looks like this:
1. data selection
2. (optional) data reduction
3. (optional) visualization

## Sea Surface Height anomaly: 1997-1998 El Nino event

### 1. Data selection

From the original dataset we extract the sea surface height (variable `zos`) and perform a geographical selection corresponding to the Pacific equator area. This greatly reduces the amount of data that will be downloaded from EDH.

In [ ]:
zos_pacific_area = ds.zos.sel(**{"latitude": slice(-5, 5), "longitude": slice(-120, -80)})
zos_pacific_area

!NB. At this point, no data has been downloaded yet, nor loaded in memory

### 2. Data reduction

We compute the mean sea surface height, in the Pacific area, over the reference period (1993-2020):

In [ ]:
zos_pacific_area_1993_2020_mean = zos_pacific_area.mean(dim="time")

After that, we can compute the sea surface height anomaly in the same area.
Calling `compute` on the result will trigger the download and computation.

We can mesure the time it takes:

In [ ]:
%%time

zos_pacific_area_anomaly = zos_pacific_area - zos_pacific_area_1993_2020_mean
zos_pacific_area_anomaly = zos_pacific_area_anomaly.compute()

We further select only the time span realative to the El Nino event, and compute the sea surface height mean:

In [ ]:
zos_pacific_area_anomaly_1997_1998 = zos_pacific_area_anomaly.sel(time=slice("1997-04", "1998-04"))
zos_pacific_area_anomaly_1997_1998_mean = zos_pacific_area_anomaly_1997_1998.mean(dim="time")
zos_pacific_area_anomaly_1997_1998_mean

### 3. Visualization
We can plot the mean sea surface height anomaly during the El Nino Event on a map.

In [ ]:
from cartopy import crs
import display
import matplotlib.pyplot as plt

In [ ]:
_, ax = plt.subplots(
    figsize=(10, 8),
    subplot_kw={"projection":  crs.Miller()},
)
zos_pacific_area_anomaly_1997_1998_mean.plot(
    ax=ax, 
    cmap="Blues", 
    transform=crs.PlateCarree(), 
    cbar_kwargs={"orientation": "horizontal", "pad": 0.05, "aspect": 40, "label": "Sea Surface Height anomaly [m]"},
)
ax.coastlines()
ax.set_title("El Nino event (1997-1998)")
plt.show()

## Other El Nino events

We want to see if other El Nino events can be found in the dataset time span. To do so, we average the sea surface height anomaly by month. Then, we further reduce it in space and plot it as a timeseries.

In [ ]:
zos_pacific_area_anomaly_monthly_mean = zos_pacific_area_anomaly.resample(time="1M").mean()
zos_pacific_area_anomaly_monthly_mean_spatial_mean = zos_pacific_area_anomaly_monthly_mean.mean(dim=["latitude", "longitude"])
zos_pacific_area_anomaly_monthly_mean_spatial_mean

In [ ]:
%%time

plt.plot(zos_pacific_area_anomaly_monthly_mean_spatial_mean['time'], zos_pacific_area_anomaly_monthly_mean_spatial_mean)
plt.xlabel('Year')
plt.ylabel('Sea Surface Height anomaly [m]')
plt.xticks(rotation=45)
plt.show()